In [ ]:
# Clone the SleepQA repo
!git clone https://github.com/IvaBojic/SleepQA.git
%cd SleepQA

Cloning into 'SleepQA'...
remote: Enumerating objects: 400, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 400 (delta 46), reused 43 (delta 19), pack-reused 313 (from 1)
Receiving objects: 100% (400/400), 31.13 MiB | 7.20 MiB/s, done.
Resolving deltas: 100% (176/176), done.
Updating files: 100% (134/134), done.
Filtering content: 100% (3/3), 1.21 GiB | 50.43 MiB/s, done.
/content/SleepQA


In [ ]:
!pip install mne pandarallel rdkit-pypi transformers accelerate polars
!git clone https://github.com/asahoo0/PyHealth.git
%cd PyHealth

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━

In [11]:
import pandas as pd
import os
from pyhealth.datasets import BaseDataset
from typing import Dict, Tuple, List

class SleepQADataset(BaseDataset):
    def __init__(self, root: str = "/content/SleepQA/data/training", dev: bool = False):
        # Load and prepare data
        corpus_path = os.path.join(root, "sleep-corpus.tsv")
        qa_path = os.path.join(root, "sleep-test.csv")

        corpus_df = pd.read_csv(corpus_path, sep="\t", header=None, names=["id", "passage", "title"])
        qa_df = pd.read_csv(qa_path, sep="\t", header=None, names=["question", "answer"])

        if dev:
            corpus_df = corpus_df.sample(n=500, random_state=42)
            qa_df = qa_df.sample(n=50, random_state=42)

        # Define tables dictionary for BaseDataset
        tables = {
            "corpus": corpus_df,
            "qa": qa_df
        }

        super().__init__(dataset_name="SleepQA", root=root, tables=tables, config_path="/content/sleepQA.yaml", dev=dev)

        # Store examples for task-level work later
        self.examples = [
            {"question": row["question"].strip(), "answer": row["answer"].strip()}
            for _, row in qa_df.iterrows()
        ]
        self.corpus = corpus_df

    def get_raw_samples(self) -> Tuple[pd.DataFrame, List[Dict[str, str]]]:
        return self.corpus, self.examples

# Main function to instantiate and test the dataset class
def main():
    dataset = SleepQADataset(dev=True)  # Set dev=True for smaller data
    corpus, examples = dataset.get_raw_samples()

    print(f"Corpus:\n{corpus.head()}")
    print(f"Examples:\n{examples[:5]}")  # Print the first 5 examples

if __name__ == "__main__":
    main()


Initializing SleepQA dataset from /content/SleepQA/data/training (dev mode: True)


INFO:pyhealth.datasets.base_dataset:Initializing SleepQA dataset from /content/SleepQA/data/training (dev mode: True)


Scanning table: corpus from /content/SleepQA/data/training/sleep-corpus.tsv


INFO:pyhealth.datasets.base_dataset:Scanning table: corpus from /content/SleepQA/data/training/sleep-corpus.tsv


Scanning table: qa from /content/SleepQA/data/training/sleep-test.csv


INFO:pyhealth.datasets.base_dataset:Scanning table: qa from /content/SleepQA/data/training/sleep-test.csv


Corpus:
        id                                            passage  \
2966  2966  heavy drinking means more than 15 drinks per w...   
6961  6965  older beds tend to sag in places, which reduce...   
3214  3215  booster shots may be given with the same vacci...   
437    437  if you go this route, you can try to sell your...   
263    263  relaxation exercises may be a component of cbt...   

                                                  title  
2966                                  alcohol and sleep  
6961              when should you replace your mattress  
3214  covid-19 vaccine: how sleep increases immune r...  
437                              how to ship a mattress  
263                                   anxiety and sleep  
Examples:
[{'question': 'what is a benefit of stomach sleeping?', 'answer': '["can help relieve snoring"]'}, {'question': 'what does good sleep during trips do?', 'answer': '["promotes wellness that lets you get the most out of any trip"]'}, {'question'